In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Vader 감성분석**

In [ ]:
import pandas as pd
text = pd.read_excel('/content/gdrive/MyDrive/EngReviews_Naver_Kakao.xlsx')
#review = text[['review', 'translated']].astype(str)
review = text['translated'].astype(str)

In [ ]:
review.head()

0                     I've been here for about 3 years
1    First of all, the gym is open 24 hours, so you...
2    Most of all, the PT is different from the regu...
3    The PT is for all the lower body, abdomen, upp...
4    First of all, the gym is open 24 hours a day, ...
Name: translated, dtype: object

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
sentiment_scores_list = []
for review_text in review:
    sentiment_scores = sia.polarity_scores(review_text)
    sentiment_scores_list.append(sentiment_scores)
sen = pd.DataFrame(sentiment_scores_list)
sen_analysis = pd.concat([text, sen],axis=1)

In [ ]:
sentiment_scores_list

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

In [ ]:
sen_analysis[['review','translated','neg','neu','pos','compound']].head()

,review,translated,neg,neu,pos,compound
0,여기 다닌지 3년정도 다닌 후기 입니다,I've been here for about 3 years,0.000,1.000,0.000,0.0000
1,우선 헬스장이 24시간이어서 언제든지 운동가능하고 항상 시원하고 쾌적해요,"First of all, the gym is open 24 hours, so you...",0.000,0.772,0.228,0.6808
2,무엇보다 피티는 일반 피티 운동기구 3-4개 하고 3세트씩 하는곳과 차원이 달라요,"Most of all, the PT is different from the regu...",0.000,1.000,0.000,0.0000
3,피티는 하체 복부 상체 일반헬스장 다 따로 하는데 여기는 같이 토탈적 으로 잦아주면...,"The PT is for all the lower body, abdomen, upp...",0.032,0.690,0.278,0.9334
4,먼저 헬스장이 24시간 열려있다보니 언제든 편하게 가서 운동을 할 수 있다는게 너무...,"First of all, the gym is open 24 hours a day, ...",0.000,0.823,0.177,0.6736


In [ ]:
sen = pd.concat([review,sen],axis = 1)
sen.head()

,translated,neg,neu,pos,compound
0,I've been here for about 3 years,0.000,1.000,0.000,0.0000
1,"First of all, the gym is open 24 hours, so you...",0.000,0.772,0.228,0.6808
2,"Most of all, the PT is different from the regu...",0.000,1.000,0.000,0.0000
3,"The PT is for all the lower body, abdomen, upp...",0.032,0.690,0.278,0.9334
4,"First of all, the gym is open 24 hours a day, ...",0.000,0.823,0.177,0.6736


**헬스사업장별 점수화**

In [ ]:
name = pd.read_excel('/content/gdrive/MyDrive/네이버카카오리뷰_1208.xlsx')

In [ ]:
name.columns

Index(['사업장명', 'URL', '리뷰'], dtype='object')

In [ ]:
result = pd.merge(sen_analysis,name,on='URL',how='left')
result = result.drop_duplicates(subset=['URL', 'review'])
result = result.drop(['리뷰'],axis = 1)
result['count'] = result.groupby('사업장명')['사업장명'].transform('count') # 사업장별 바이그램개수
result['점수'] = result['compound']/result['count'] # 문장단위 점수화
result_df = result.groupby('사업장명', as_index=False)['점수'].sum() #사업장별 점수화
result_df.head()

,사업장명,점수
0,24시PT앤태닝,0.398360
1,86피티,0.359450
2,ACUB휘트니스,0.275145
3,BNB PT 스튜디오,0.440400
4,F45 강남점,0.462004


In [ ]:
### 피피티용 코드

In [ ]:
result = pd.merge(sen_analysis,name,on='URL',how='left')

result = result.drop_duplicates(subset=['URL', 'review'])

result = result.drop(['리뷰'],axis = 1)

result['count'] = result.groupby('사업장명')['사업장명'].transform('count')

result['점수'] = result['compound']/result['count']

result[['사업장명','compound','count','점수']].head(10)

,사업장명,compound,count,점수
0,24시PT앤태닝,0.0000,20.0,0.000000
5,24시PT앤태닝,0.6808,20.0,0.034040
10,24시PT앤태닝,0.0000,20.0,0.000000
15,24시PT앤태닝,0.9334,20.0,0.046670
20,24시PT앤태닝,0.6736,20.0,0.033680
25,24시PT앤태닝,0.4927,20.0,0.024635
30,24시PT앤태닝,0.6209,20.0,0.031045
35,24시PT앤태닝,0.2484,20.0,0.012420
40,24시PT앤태닝,0.0000,20.0,0.000000
45,24시PT앤태닝,0.6369,20.0,0.031845


In [ ]:
result_df.to_excel('/content/gdrive/MyDrive/헬스장별긍부정점수1213.xlsx')

**카테고리별 정규화(0~10)**

In [ ]:
import pandas as pd
review_count = pd.read_excel('/content/gdrive/MyDrive/1212_야근_점수화.xlsx')
review_count.head()

,사업장명,카테고리,빈도,바이그램수,빈도/바이그램수
0,24시PT앤태닝,수업(PT),26,90,0.288889
1,24시PT앤태닝,기구,12,90,0.133333
2,24시PT앤태닝,시설,4,90,0.044444
3,24시PT앤태닝,서비스,2,90,0.022222
4,86피티,수업(PT),2,12,0.166667


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 10)) #0과 10사이로 정규화

In [ ]:
for i in review_count['카테고리'] :
  selected_rows = review_count[review_count['카테고리'] == i]
  normalized_values = scaler.fit_transform(selected_rows['빈도/바이그램수'].values.reshape(-1, 1))
  review_count.loc[review_count['카테고리'] == i, '정규화'] = normalized_values.flatten()

,사업장명,카테고리,빈도,바이그램수,빈도/바이그램수,정규화
0,24시PT앤태닝,수업(PT),26,90,0.288889,4.839604
1,24시PT앤태닝,기구,12,90,0.133333,2.572060
2,24시PT앤태닝,시설,4,90,0.044444,0.713028
3,24시PT앤태닝,서비스,2,90,0.022222,0.133695
4,86피티,수업(PT),2,12,0.166667,2.607298


In [ ]:
review_count.head()

,사업장명,카테고리,빈도,바이그램수,빈도/바이그램수,정규화
0,24시PT앤태닝,수업(PT),26,90,0.288889,4.839604
1,24시PT앤태닝,기구,12,90,0.133333,2.572060
2,24시PT앤태닝,시설,4,90,0.044444,0.713028
3,24시PT앤태닝,서비스,2,90,0.022222,0.133695
4,86피티,수업(PT),2,12,0.166667,2.607298


In [ ]:
review_count[['사업장명','카테고리','빈도','바이그램수','정규화']].head()

,사업장명,카테고리,빈도,바이그램수,정규화
0,24시PT앤태닝,수업(PT),26,90,4.839604
1,24시PT앤태닝,기구,12,90,2.572060
2,24시PT앤태닝,시설,4,90,0.713028
3,24시PT앤태닝,서비스,2,90,0.133695
4,86피티,수업(PT),2,12,2.607298


추천 점수화

In [ ]:
result_category = pd.read_excel('/content/gdrive/MyDrive/1213카테고리별로점수정규화.xlsx')
result_posneg = pd.read_excel('/content/gdrive/MyDrive/헬스장별긍부정점수1213.xlsx')
result_discount = pd.read_excel('/content/gdrive/MyDrive/진실리뷰용_리뷰이벤트.xlsx')
result_false = pd.read_excel('/content/gdrive/MyDrive/허위리뷰.xlsx')

데이터 전처리 및 합치기

In [ ]:
result_posneg.rename(columns = {'점수':'긍부정점수',},inplace=True)
result_discount.rename(columns = {'할인율':'리뷰이벤트할인율',},inplace=True)
result_false.rename(columns = {'허위리뷰 비율':'허위리뷰비율',},inplace=True)
result_discount = result_discount.drop(['네이버홈 URL','이벤트 출처','이벤트 종류', '진행 상황', '정가','이벤트 할인가', '할인 금액','증정품', '이벤트 상세'],axis = 1)
result_false = result_false.drop(['리뷰ID', '리뷰', '글자수', 'URL', '허위리뷰수', '네이버 리뷰수', '카카오 리뷰수',
       '합산 리뷰수(잠백이 미포함)', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],axis = 1)
result_posneg = result_posneg.drop(['Unnamed: 0'],axis = 1)
rec_sentence = pd.merge(result_category,result_posneg,on='사업장명',how='left')
rec_sentence = pd.merge(rec_sentence,result_discount,on='사업장명',how='left')
rec_sentence = pd.merge(rec_sentence,result_false,on='사업장명',how='left')
# 할인율/허위리뷰비율/긍부점점수 null 값 > 0으로 바꾸기
rec_sentence['리뷰이벤트할인율'] = rec_sentence['리뷰이벤트할인율'].fillna(0)
rec_sentence['허위리뷰비율'] = rec_sentence['허위리뷰비율'].fillna(0)
rec_sentence['긍부정점수'] = rec_sentence['긍부정점수'].fillna(0)

In [ ]:
rec_sentence.head()

,사업장명,카테고리,빈도,바이그램수,빈도/바이그램수,정규화,긍부정점수,리뷰이벤트할인율,허위리뷰비율
0,24시PT앤태닝,수업(PT),26,90,0.288889,4.839604,0.39836,0.0,0.0
1,24시PT앤태닝,기구,12,90,0.133333,2.572060,0.39836,0.0,0.0
2,24시PT앤태닝,시설,4,90,0.044444,0.713028,0.39836,0.0,0.0
3,24시PT앤태닝,서비스,2,90,0.022222,0.133695,0.39836,0.0,0.0
4,86피티,수업(PT),2,12,0.166667,2.607298,0.35945,0.0,0.0


추천 점수화
- 공식 : 정규화*(1+긍부정점수-리뷰이벤트할인율-허위리뷰비율)

In [ ]:
가중치 = (1-rec_sentence['리뷰이벤트할인율']-rec_sentence['허위리뷰비율']+rec_sentence['긍부정점수'])

rec_sentence['추천점수'] = 가중치*rec_sentence['정규화']

In [ ]:
rec_sentence.head()

,사업장명,카테고리,빈도,바이그램수,빈도/바이그램수,정규화,긍부정점수,리뷰이벤트할인율,허위리뷰비율,추천점수
0,24시PT앤태닝,수업(PT),26,90,0.288889,4.839604,0.39836,0.0,0.0,6.767509
1,24시PT앤태닝,기구,12,90,0.133333,2.572060,0.39836,0.0,0.0,3.596666
2,24시PT앤태닝,시설,4,90,0.044444,0.713028,0.39836,0.0,0.0,0.997070
3,24시PT앤태닝,서비스,2,90,0.022222,0.133695,0.39836,0.0,0.0,0.186953
4,86피티,수업(PT),2,12,0.166667,2.607298,0.35945,0.0,0.0,3.544491


In [ ]:
rec_sentence.to_excel('/content/gdrive/MyDrive/추천점수최종.xlsx')